https://maap-ops-dataset.s3.amazonaws.com/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-1000.zip.tif
s3://maap-ops-dataset/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-1000.zip.tif

In [ ]:
#for zip in glob
# unzip renaming tif inside to match zip tilename
# better idea rio cogeo /vsizip/nameof.zip nameof.tif
!cd /projects/tmp/gee
!less todo.txt | parallel --bar -j 10 ./maketif.sh {}

In [2]:
!pip install cogeo-mosaic

  Using cached cogeo_mosaic-3.0.0-py3-none-any.whl
  Using cached supermercado-0.2.0-py3-none-any.whl (8.1 kB)
  Using cached pygeos-0.10-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (2.0 MB)
  Using cached cachetools-4.2.2-py3-none-any.whl (11 kB)
  Using cached mercantile-1.2.1-py3-none-any.whl (14 kB)
  Using cached rio_tiler-2.1.0-py3-none-any.whl
  Using cached pystac-0.5.6-py3-none-any.whl (136 kB)
  Using cached rio_color-1.0.2-cp37-cp37m-manylinux2010_x86_64.whl (196 kB)
  Using cached importlib_resources-5.1.4-py3-none-any.whl (26 kB)
  Using cached numexpr-2.7.3-cp37-cp37m-manylinux2010_x86_64.whl (471 kB)
  Using cached rio_mucho-1.0.0-py3-none-any.whl (5.8 kB)
  Attempting uninstall: pygeos
    Found existing installation: pygeos 0.8
    Uninstalling pygeos-0.8:
      Successfully uninstalled pygeos-0.8
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [1]:
from glob import glob
import os

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

In [6]:
def local_to_s3(url):
    return url.replace("/projects/maap-users", "s3://maap-ops-dataset/maap-users")
    
def s3_to_https(url):
    parts = url.split("/")
    return os.path.join('https://',f'{parts[2]}.s3.amazonaws.com', *parts[3:])

In [18]:
#glob find the tifs
tif_dir = "/projects/maap-users/alexdevseed/gee/"

local_tiles = glob(os.path.join(tif_dir,"*class*.zip.tif"), recursive=False)
tiles = [local_to_s3(tile) for tile in local_tiles]

In [20]:
len(tiles)

2304

In [21]:

#tiles = [
#    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30542_comp_cog_2015-2020_dps.tif',
#    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30543_comp_cog_2015-2020_dps.tif',
#    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30822_comp_cog_2015-2020_dps.tif',
#    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30823_comp_cog_2015-2020_dps.tif',
#]


mosaicdata = MosaicJSON.from_urls(tiles, minzoom=8, maxzoom=16)

/opt/conda/lib/python3.7/site-packages/rio_tiler/io/cogeo.py:117: NoOverviewWarning: The dataset has no Overviews. rio-tiler performances might be impacted.
  NoOverviewWarning,
'/vsis3/maap-ops-dataset/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-1023.zip.tif' not recognized as a supported file format.
'/vsis3/maap-ops-dataset/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-1103.zip.tif' not recognized as a supported file format.
'/vsis3/maap-ops-dataset/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-1139.zip.tif' not recognized as a supported file format.
'/vsis3/maap-ops-dataset/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-1246.zip.tif' not recognized as a supported file format.
'/vsis3/maap-ops-dataset/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-1251.zip.tif' not recognized as a supported file format.
'/vsis3/maap-ops-dataset/maap-users/alexdevseed/gee/TTE_DATA_v201902v2-class-1281.zip.tif' not recognized as a supported file format.
'/vsis3/maap-ops-d

In [22]:
mosaic_json = "/projects/maap-users/alexdevseed/gee/forest_classificiation.json"
with MosaicBackend(mosaic_json, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)

In [5]:
import ipycmc

In [7]:
wmts_url = "https://baxpil3vd6.execute-api.us-east-1.amazonaws.com/mosaicjson/WMTSCapabilities.xml"
params = [
    "tile_format=png",
    "tile_scale=1",
    "pixel_selection=first",
    "TileMatrixSetId=WebMercatorQuad",
    "url=s3%3A%2F%2Fmaap-ops-dataset%2Fmaap-users%2Falexdevseed%2Fgee%2Fforest_classificiation.json",
    "bidx=1",
    "resampling_method=nearest",
    "return_mask=true",
    "colormap_name=viridis"
]
#TODO url encode so the params can be human readable
wmts_call = "?".join([wmts_url, "&".join([*params])])
wmts_call

'https://baxpil3vd6.execute-api.us-east-1.amazonaws.com/mosaicjson/WMTSCapabilities.xml?tile_format=png&tile_scale=1&pixel_selection=first&TileMatrixSetId=WebMercatorQuad&url=s3%3A%2F%2Fmaap-ops-dataset%2Fmaap-users%2Falexdevseed%2Fgee%2Fforest_classificiation.json&bidx=1&resampling_method=nearest&return_mask=true&colormap_name=viridis'

In [ ]:
w = ipycmc.MapCMC()
w

In [ ]:
# this layer is crashing CMC
#w.load_layer_config(wmts_call
#                    , "wmts/xml")